In [12]:
import numpy as np
import model
from importlib import reload
reload(model)


<module 'model' from 'C:\\Users\\Paul\\Documents\\Keras-progressive_growing_of_gans\\Progressive growing of GANs\\model.py'>

In [15]:
G = model.Generator()

In [52]:
from keras.utils import plot_model
plot_model(G)

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [49]:
from keras.models import Model
model = G

layers = [l for l in model.layers]

x = layers[0].output
for i in range(1, len(layers)):
    if 'BN' in layers[i].name:
        print('here')
        x = BatchNormalization(name=layer.name + '_new')(x)
    else:
        print(layers[i].name)
        x = layers[i](x)

new_model = Model(input=layers[0].input, output=x)



Gnorm
G1nb
G1aPad
G1a
G1aWS
add_bias_layer_121
activation_121
here
G1aPN
G1bPad
G1b
G1bWS
add_bias_layer_122
activation_122
here
G1bPN
G2up
G2aPad
G2a
G2aWS
add_bias_layer_123
activation_123
here
G2aPN
G2bPad
G2b
G2bWS
add_bias_layer_124
activation_124
here
G2bPN
G3up
G3aPad
G3a
G3aWS
add_bias_layer_125
activation_125
here
G3aPN
G3bPad
G3b
G3bWS
add_bias_layer_126
activation_126
here
G3bPN
G4up
G4aPad
G4a
G4aWS
add_bias_layer_127
activation_127
here
G4aPN
G4bPad
G4b
G4bWS
add_bias_layer_128
activation_128
here
G4bPN
Glod0NIN
Glod1NIN


ValueError: Input 0 is incompatible with layer Glod1NIN: expected axis -1 of input shape to have value 256 but got shape (None, 32, 32, 1)

In [39]:
import re
from keras.models import Model

# source : https://stackoverflow.com/questions/49492255/how-to-replace-or-insert-intermediate-layer-in-keras-model

#def insert_layer_nonseq(model, layer_regex, insert_layer_factory, insert_layer_name=None, position='after'):
model = G

# Auxiliary dictionary to describe the network graph
network_dict = {'input_layers_of': {}, 'new_output_tensor_of': {}}

# Set the input layers of each layer
for layer in model.layers:
    for node in layer.outbound_nodes:
        layer_name = node.outbound_layer.name
        if layer_name not in network_dict['input_layers_of']:
            network_dict['input_layers_of'].update(
                    {layer_name: [layer.name]})
        else:
            network_dict['input_layers_of'][layer_name].append(layer.name)

# Set the output tensor of the input layer
network_dict['new_output_tensor_of'].update(
        {model.layers[0].name: model.input})
#print(network_dict)

# Iterate over all layers after the input
for layer in model.layers[1:]:

    # Determine input tensors
    print(network_dict['input_layers_of'][layer.name])
    layer_input = [network_dict['new_output_tensor_of'][layer_aux] 
            for layer_aux in network_dict['input_layers_of'][layer.name]]
    if len(layer_input) == 1:
        layer_input = layer_input[0]

    # Insert layer if name matches the regular expression
    if 'BN' in layer.name:
        # replace layer
        x = layer_input

        new_layer = BatchNormalization(name=layer.name + '_new')
        
        x = new_layer(x)
        print('Layer {} inserted after layer {}'.format(new_layer.name, layer.name))

    else:
        print(layer.name)
        print(layer_input)
        x = layer(layer_input)

    # Set new output tensor (the original one, or the one of the inserted
    # layer)
    network_dict['new_output_tensor_of'].update({layer.name: x})

#return Model(inputs=model.inputs, outputs=x)

['Glatents', 'Glatents']
Gnorm
[<tf.Tensor 'Glatents_10:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'Glatents_10:0' shape=(?, 256) dtype=float32>]


TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [22]:
for l in G.layers:
    if 'BN' in l.name:
        print(l.get_config())
        print()
        Bslayer = BatchNormalization(name=l.name + '_new')
        l = Bslayer

{'name': 'G1aBN', 'trainable': True, 'axis': -1, 'momentum': 0.99, 'epsilon': 0.001, 'center': True, 'scale': True, 'beta_initializer': {'class_name': 'Zeros', 'config': {}}, 'gamma_initializer': {'class_name': 'Ones', 'config': {}}, 'moving_mean_initializer': {'class_name': 'Zeros', 'config': {}}, 'moving_variance_initializer': {'class_name': 'Ones', 'config': {}}, 'beta_regularizer': None, 'gamma_regularizer': None, 'beta_constraint': None, 'gamma_constraint': None}

{'name': 'G1bBN', 'trainable': True, 'axis': -1, 'momentum': 0.99, 'epsilon': 0.001, 'center': True, 'scale': True, 'beta_initializer': {'class_name': 'Zeros', 'config': {}}, 'gamma_initializer': {'class_name': 'Ones', 'config': {}}, 'moving_mean_initializer': {'class_name': 'Zeros', 'config': {}}, 'moving_variance_initializer': {'class_name': 'Ones', 'config': {}}, 'beta_regularizer': None, 'gamma_regularizer': None, 'beta_constraint': None, 'gamma_constraint': None}

{'name': 'G2aBN', 'trainable': True, 'axis': -1, 'mo